In [2]:
import pymc3 as pm
import numpy as np

import theano
import theano.tensor as tt
import matplotlib.pyplot as plt

# define a theano Op (Operator/Operation?) for our likelihood function
class LogLike(tt.Op):
    itypes = [tt.dvector] # expects vector of parameter values
    otypes = [tt.dscalar] # outputs a single scalar (loglik)
    
    def __init__(self, loglike, data, x, sigma):
        """
        Initialise the Op with various things that our log-likelihood
        function requires. below are the things that are needed in this 
        particular example.
        
        Parameters
        ----------
        loglike:
            The log-likelihood function we've defined
        data:
            The "observed" data that our log-likelihood functions takes in
        x:
            the independent variable that our models requires
        sigma:
            The noise standard deviation that our function requires
        """ 
        # inputs as class attributes
        self.likelihood = loglike
        self.data = data
        self.x = x
        self.sigma = sigma
    
    def perform(self, node, inputs, outputs):
        # method which is used when calling the Op
        theta = inputs
        
        # call log-likelihood function
        logl = self.likelihood(theta, self.x, self.data, self.sigma)
        print(logl)
        
        outputs[0][0] = np.array(logl)

def my_model(theta, x):
    m, c = theta[0]
    #c = theta[1]
    return m * x + c

def my_loglike(theta, x, data, sigma):
    """
    custom log likelihood function that takes in parameters
    
    Paramerters
    -----------
    theta : model parameters
    x : independent variable
    data : dependent variable
    sigma : should actually be in theta but is assumed to be fixed for now
    """ 
    model = my_model(theta, x) # assumed dgp given parameters theta (much like simulate funciton)
    
    return -(0.5 / sigma ** 2) * np.sum((data - model) ** 2)

N = 200
sigma = 1.
x = np.linspace(0., 9., N)

mtrue = .4
ctrue = 3.

truemodel = my_model([[mtrue, ctrue]], x)

data = sigma * np.random.randn(N) + truemodel

ndraws = 3000 # number of draws from the distribution 
nburn = 1000 # number of burn in points 

# create our Op
logl = LogLike(my_loglike, data, x, sigma)

# use PyMC3 to sample from log-likelihood
with pm.Model():
    # uniform priors on m and c
    m = pm.Uniform('m', lower=-10., upper=10.)
    c = pm.Uniform('c', lower=-10., upper=10.)
    
    # convert m and c to a tensor vector
    theta = tt.as_tensor_variable((m, c))  
    
    # use DensityDist and lambda function 
    pm.DensityDist('likelihood', lambda v: logl(v), observed={'v': theta})
    
    trace = pm.sample(ndraws, tune=nburn, discard_tuned_samples=True)

pm.traceplot(trace, lines=[('m', {}, [mtrue]), ('c', {}, [ctrue])] )

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Initializing NUTS failed. Falling back to elementwise auto-assignment.


-2430.6683526409847
-2430.6683526409847
-2430.6683526409847


Multiprocess sampling (2 chains in 2 jobs)
CompoundStep
>Slice: [c]
>Slice: [m]
Sampling 2 chains:   0%|          | 0/8000 [00:00<?, ?draws/s]

-2430.6683526409847
-2430.6683526409847
-8190.017500627957
-219.73106147848983
-1067.7230216363732
-2110.196689688587
-2656.9631546854976
-2303.0111258497755
-2303.0111258497755
-34310.51327981518
-76625.03403175483
-2883.5396478016073
-6250.004167537196
-1713.178595339979
-219.73106147848983
-1713.178595339979
-1067.7230216363732
-2110.196689688587
-13465.715550574141
-2656.9631546854976
-2302.9089056070807
-1041.1899536573703
-876.5282031318026
-1041.1899536573703
-876.5282031318026
-1423.2639902164283
-21353.755115876535
-38925.7483934392
-68302.16199936619
-12264.743818971816
-53215.12361046568
-5468.889699389288
-3430.731328705379
-9096.125313897819
-5828.926999879177
-664.5472109635457
-801.4382612355963
-664.5472109635457
-1256.8485015013562
-801.4382612355963
-1935.0129777868096
-12971.33006233756
-1918.61380245409
-1870.1952212496449
-2388.700061807233
-897.7692708901207
-338.81348526039983
-1662.0519706810676
-338.81348526039983
-31568.856053580203
-1156.4763273400765
-31039.

Sampling 2 chains:   0%|          | 4/8000 [00:00<03:20, 39.83draws/s]

...

Sampling 2 chains: 100%|█████████▉| 7982/8000 [03:35<00:00, 28.43draws/s]

-116.13484842487392
-149.58615081908894
-215.3285846367741
-136.9580954269488
-131.69556233491642
-120.85599518859424
-114.99573320142292
-114.99573320142292
-118.14651389673959
-211.00928307293063
-115.76897709348316
-149.64393088026242
-118.63195292478197
-116.66905815475641
-115.48556638334433
-115.08641102974833
-115.08641102974833
-131.858305681746
-236.58710162437694
-116.9615315269867
-116.9615315269867
-133.65469742593905
-192.80530465395663
-125.8108581150987
-118.45146187164164
-117.39637568048158
-118.12946126798394
-117.50484072250262
-116.33935111640312
-116.33935111640312
-180.42671251385013
-224.94305775216824
-128.37396556901473
-128.36882980825052
-122.03799683482865
-117.70986393983125
-116.9704250679915
-115.90468621028975
-115.90468621028975
-115.93359781227431
-216.42825266890443
-215.48589985761575
-157.0977767091209
-118.08546105661657
-121.19569623412474
-117.2013099216883
-116.25044685252755


Sampling 2 chains: 100%|█████████▉| 7985/8000 [03:35<00:00, 27.62draws/s]

-116.25044685252755
-135.10777338666733
-241.00259624187578
-232.40240761997717
-169.27196965094817
-156.94650082078482
-116.25639674751712
-116.25639674751712
-180.35180326990718
-217.92862998854764
-177.74386910296738
-117.32947492588201
-117.32947492588201
-176.8331169655977
-204.30851842820636
-138.84544866519605
-118.10292037171543
-116.85088796198832
-116.85088796198832
-120.7827069862278
-187.7719060093384
-157.54929809125673
-127.93872332520912
-118.137302584812
-117.31061299095603
-115.75766692367216
-115.75766692367216
-220.40253079079724
-227.42881759806784
-139.79902223486926
-126.41219275633344
-119.88540137211388
-116.35063727886175
-116.35063727886175
-199.07090832868266
-201.7644909579032
-151.9150559747838
-128.4625310974453
-127.35751221411809
-128.06527325668748
-116.47642773786924


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

